PART 1 - EXTRACT, TRANSFORM AND LOAD PROCESS

In [5]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q
!pip install numpy

  Using cached numpy-2.1.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.1.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)


In [6]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler


In [12]:
#Create a SparkSession

spark = SparkSession.builder.appName("spark_ml").getOrCreate()

24/10/28 20:25:09 WARN Utils: Your hostname, thiago-Dell-G15-5530 resolves to a loopback address: 127.0.1.1; using 192.168.15.7 instead (on interface wlp0s20f3)
24/10/28 20:25:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/28 20:25:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
#Download the data file

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv

--2024-10-28 20:17:20--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolvendo cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Conectando-se a cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 60682 (59K) [text/csv]
Salvando em: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59,26K   160KB/s    em 0,4s    

2024-10-28 20:17:22 (160 KB/s) - ‘NASA_airfoil_noise_raw.csv’ salvo [60682/60682]



In [13]:
# Load the dataset that you have downloaded in the previous task

df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True, inferSchema=True)


In [14]:
df.show(5)


+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



In [15]:
rowcount1 = df.count()
print(rowcount1)

1522


In [17]:
#Drop all the duplicate rows from the dataset

df = df.dropDuplicates()


1503


In [18]:
rowcount2 = df.count()
print(rowcount2)

1503


In [19]:
#Drop all the rows that contain null values from the dataset

df = df.dropna()


In [20]:
rowcount3 = df.count()
print(rowcount3)

1499


In [21]:
#Rename the column "SoundLevel" to "SoundLevelDecibels"

df = df.withColumnRenamed("SoundLevel", "SoundLevelDecibels")


In [22]:
#Save the dataframe in parquet format, name the file as "NASA_airfoil_noise_cleaned.parquet"

df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")


In [23]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


PART 2 - CREATE A MACHINE LEARNING PIPELINE

In [24]:
#Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe

df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")


In [32]:
rowcount4 = df.count()
print(rowcount4)


1499


In [25]:
#Define the VectorAssembler pipeline stage

assembler = VectorAssembler(inputCols=['Frequency','AngleOfAttack','ChordLength','FreeStreamVelocity','SuctionSideDisplacement'], outputCol="features")


In [26]:
#Define the StandardScaler pipeline stage

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")


In [27]:
#Define the Model creation pipeline stage
#Create a LinearRegression stage to predict "SoundLevelDecibels"

lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")

In [28]:
#Build the pipeline

pipeline = Pipeline(stages=[assembler, scaler, lr])


In [29]:
#Split the data

(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)


In [30]:
# Fit the pipeline using the training data

pipelineModel = pipeline.fit(trainingData)


/home/thiago/.pyenv/versions/3.12.6/lib/python3.12/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/thiago/.pyenv/versions/3.12.6/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
24/10/28 20:37:41 WARN Instrumentation: [d554b137] regParam is zero, which might cause numerical instability and overfitting.
24/10/28 20:37:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/28 20:37:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/10/28 20:37:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [33]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


PART 3 - EVALUATE THE MODEL


In [34]:
#Predict using the model

predictions = pipelineModel.transform(testingData)



/home/thiago/.pyenv/versions/3.12.6/lib/python3.12/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/thiago/.pyenv/versions/3.12.6/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [35]:
#Print the MSE

from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

24.99766625502418


In [36]:
#Print the MAE

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)


3.9136790958812044


In [37]:
#Print the R-Squared(R2)

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.4959688408974623


In [38]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  25.0
Mean Absolute Error =  3.91
R Squared =  0.5
Intercept =  132.88


PART 4 - PERSIST THE MODEL

In [39]:
#Save the model to the path "spark_ml_nasa"

pipelineModel.write().save("spark_ml_nasa")

In [40]:
# Load the pipeline model you have created in the previous step

loadedPipelineModel = PipelineModel.load("spark_ml_nasa")

In [41]:
# Use the loaded pipeline model and make predictions using testingData

predictions = loadedPipelineModel.transform(testingData)

/home/thiago/.pyenv/versions/3.12.6/lib/python3.12/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/thiago/.pyenv/versions/3.12.6/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [42]:
#Show the predictions

predictions.select("SoundLevelDecibels","prediction").show()

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           128.679|122.59722914376778|
|            133.42|127.37968204568838|
|           119.146|130.34077425074506|
|           116.074|131.11016975113537|
|           134.319|127.12627360125096|
|            125.01|127.89456373905155|
|           125.941|131.06220981224084|
|           130.588|125.73739953848445|
|           128.354|121.53249832197925|
|           121.783|124.20059665619313|
|            122.94|125.87997778533571|
|           116.146|125.24362112904095|
|           114.044|126.06429872612995|
|           109.951|127.67830278943778|
|           125.974|121.25022147564815|
|           116.066|123.31966959832609|
|           118.595|124.20046348885936|
|           126.395| 126.1606883964179|
|           130.089|122.53378592206057|
|           131.889|123.42922049990014|
+------------------+------------------+
only showing top 20 rows



In [43]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.9906
Coefficient for AngleOfAttack is -2.2881
Coefficient for ChordLength is -3.3269
Coefficient for FreeStreamVelocity is 1.4832
Coefficient for SuctionSideDisplacement is -2.0551


In [44]:
#Stop Spark Session

spark.stop()